In [1]:
using NamedColors
using Colors
using ColorTypes
using StatsBase

In [2]:
training_dict = load_xkcd();

In [3]:
function encode_output(raw::Colorant)
    #Encoding Angles: http://stats.stackexchange.com/a/218547/36769
    hsv = convert(HSV, raw)
    [(sind(hsv.h)+1)/2, (cosd(hsv.h)+1)/2, hsv.s, hsv.v]
end

function decode_output(out)
    hue = rad2deg(atan2(2out[1]-1, 2out[2]-1))
    sat = out[3]
    val = out[4]
    HSV(hue, sat, val)
end

y1 = encode_output(y_raw_1)
@show y1
decode_output(y1)

LoadError: UndefVarError: y_raw_1 not defined

In [4]:
function char2index(c::Char)::Int
    if c ∈ 'a':'z'
        c-'a'+1
    elseif c==' '
        'z'+1+1
    elseif c=='/'
        'z'+2+1
    elseif c=='\''
        'z'+3+1
    else
        error("$c is out of range")
    end
end
function encode_input(name::AbstractString)
    name_inds::Vector{Int} = char2index.(collect(name))
    counts(name_inds, 26+3)./length(name)
end

encode_input (generic function with 1 method)

In [5]:
using Learn
import MNIST
using MLDataUtils
using StatsBase
using StochasticOptimization.Iteration

In [6]:
function get_training(training_dict)
    colors = collect(values(training_dict))
    y_train = hcat(encode_output.(colors)...)
    
    names = collect(keys(training_dict))
    x_train = hcat(encode_input.(names)...)
    x_train, y_train
end


get_training (generic function with 1 method)

In [7]:
train = get_training(training_dict)

(
[0.0909091 0.0 … 0.166667 0.0588235; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0],

Float32[0.994721 0.350743 … 0.230551 0.995183; 0.427534 0.977203 … 0.0788147 0.430766; 0.929032 0.892019 … 0.958333 0.595142; 0.607843 0.835294 … 0.376471 0.968627])

In [18]:
function my_test_loss(obj, testdata)
    totloss = 0.0
    totcorrect = 0
    for (x,y) in each_obs(testdata)
        totloss += transform!(obj,y,x)
    end
    totloss
end


tracer = IterFunction((obj, i) -> begin
    if mod1(i,1000)==1000
        totloss  = my_test_loss(obj, train, 500)
        println(i, " totloss:  ", totloss)
    end
end)

StochasticOptimization.IterFunction(#10,1)

In [29]:
nin, nh, nout = size(train[1],1), [4^4,4^3], size(train[2],1)
t = nnet(nin, nout, nh, :softplus, :logistic)
obj = objective(t, ElasticNetPenalty(1e-5))

ObjectiveFunctions.RegularizedObjective{Transformations.Chain{Float64,Transformations.Params{SubArray{Float64,1,Array{Float64,1},Tuple{UnitRange{Int64}},true},Tuple{},Tuple{}}},ObjectiveFunctions.LossTransform{Float64,LossFunctions.CrossentropyLoss},Penalties.ElasticNetPenalty{Float64}}(Chain{Float64}(
   LayerNorm{29-->256, mu=0.0, sigma=1.0}
   softplus{256}
   LayerNorm{256-->64, mu=0.0, sigma=1.0}
   softplus{64}
   LayerNorm{64-->4, mu=0.0, sigma=1.0}
   logistic{4}
) ,ObjectiveFunctions.LossTransform{Float64,LossFunctions.CrossentropyLoss}(LossFunctions.CrossentropyLoss(),4,input(4,),target(4,),output(1,)),Penalties.ElasticNetPenalty{Float64}(1.0e-5,0.5))

In [30]:
learner = make_learner(
    GradientLearner(0.01, Adadelta()),
    #tracer,
    maxiter = 10_000
)
learn!(obj, learner, InfiniteBatches(train, size=5))

In [31]:
function get_colour(obj, name)
    transform!(obj,[0,0,0,0f0],encode_input(name))
    decode_output( output_value(obj.transformation ))
end
function display_colour(obj,name)
    display(name)
    display(get_colour(obj, name))
    display("-"^16)
end

display_colour (generic function with 1 method)

In [32]:
display_colour(obj, "green")
display_colour(obj, "red")
display_colour(obj, "lightred")
display_colour(obj, "dirtyred")
display_colour(obj, "lightredishblue")
display_colour(obj, "bluish red")
display_colour(obj, "blue")
display_colour(obj, "light blue")
display_colour(obj, "lightred")



"green"

"----------------"

"red"

"----------------"

"lightred"

...

"----------------"

"light blue"

"----------------"

"lightred"

"----------------"

In [33]:
display_colour(obj, "hot pink")
display_colour(obj, "hot blue")
display_colour(obj, "hot green")
display_colour(obj, "dark blue")
display_colour(obj, "light black")
display_colour(obj, "black")
display_colour(obj, "dark black")

"hot pink"

"----------------"

"hot blue"

"----------------"

"hot green"

...

"----------------"

"black"

"----------------"

"dark black"

"----------------"

In [35]:
Pkg.clone("https://github.com/benmoran/TensorFlow.jl")

INFO: Cloning TensorFlow from https://github.com/benmoran/TensorFlow.jl
INFO: Computing changes...
INFO: No packages to install, update or remove
INFO: Package database updated
